In [37]:
!pip install  opencv-contrib-python==4.5.3.56
import numpy as np
import cv2 
import scipy
import matplotlib.pylab as plt
from skimage import io

You should consider upgrading via the 'C:\Users\RATeL\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [38]:
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

In [39]:
def preprocessing(img) :
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return (img, img_gray)

In [40]:
def feature_object_detection(template_img, template_gray, query_vid, query_gray, min_match_number) :
    template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.7*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number :
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 2) # H RANSAC
        # get the bounding box around template image
        h, w = template_img.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
        transformed_box = cv2.perspectiveTransform(template_box, H)

        detected_vid = cv2.polylines(query_vid, [np.int32(transformed_box)], True, (0,0,255), 2, cv2.LINE_AA)
        drawmatch_vid = cv2.drawMatchesKnn(template_img, template_kpts, detected_vid, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

        return detected_vid, drawmatch_vid
    else :
        return

In [51]:
vid = cv2.VideoCapture('./videos/final_exam/Dataset-1/left_output-1.avi')

while vid.isOpened() :
    ret, frame = vid.read()

        
    if ret :

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        template_img = cv2.imread('./images/final_exam/Templates/Template-1.png')
        template_img, template_gray = preprocessing(template_img) 
        query_vid, query_gray = preprocessing(frame)

        kernel = np.array([[0,0,0],[0,1.5,0],[0,0,0]],dtype=np.float32)
        query_gray = cv2.filter2D(frame_gray,-3,kernel)
        
        feature_object_detection(template_img, template_gray, query_vid, query_gray, 14)
        cv2.imshow('Query more like Q myself', frame)

        if cv2.waitKey(int(1000/30)) & 0xFF == ord('q') : # this line control the period between image frame
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()